<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/sobert_intent_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/pj/intent/intents_dataset_train.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/pj/intent/intents_dataset_valid.csv',
    split='train'
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset_train)

Dataset({
    features: ['type', 'id', 'title', 'description', 'description_raw', 'code', 'code_fea', 'Concat_Text', 'intents'],
    num_rows: 705
})


In [ ]:
classes = ['Discrepancy', 'Errors', 'Review', 'Conceptual', 'Learning', 'How-to', 'Other']
class2id = {'Discrepancy' : 0, 'Errors' : 1, 'Review' : 2, 'Conceptual' : 3, 'Learning' : 4, 'How-to' : 5, 'Other' : 6}
id2class = {0 : 'Discrepancy', 1 : 'Errors', 2 : 'Review', 3 : 'Conceptual', 4: 'Learning', 5: 'How-to', 6 :'Other'}

In [ ]:
from transformers import AutoTokenizer

model_path = 'mmukh/SOBertBase'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(example):
   all_labels = example['intents'].split(' ')
   labels = [0. for i in range(len(classes))]
   for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.

   example = tokenizer(example['Concat_Text'], truncation=True, return_tensors="pt")
   example['labels'] = labels
   example['input_ids'] = example['input_ids'].squeeze(0)
   example['token_type_ids'] = example['token_type_ids'].squeeze(0)
   example['attention_mask'] = example['attention_mask'].squeeze(0)
   return example

tokenized_train_dataset = dataset_train.map(preprocess_function)
tokenized_valid_dataset = dataset_valid.map(preprocess_function)

Map:   0%|          | 0/705 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset[0]

{'type': "['Conceptual']",
 'id': 'https://serverfault.com/questions/1070165',
 'title': 'Did Domino 10 change NDX file names?',
 'description': 'I\'ve just upgraded a server from IBM Domino 9.0.1 to Domino 10.0.1, and am having some trouble with NDX files (view indexes stored outside databases). I tested some custom app functionality after the upgrade, and found that code which used views sometimes produced the error message "Attempt to Reopen an Open Container". To resolve the error, I ran the server command load updall -R to rebuild all used views. After that completed, I found there are now two NDX files for every NIFNSF-enabled database. For each NSF, the older NDX file was last updated before the upgrade, and is named to match the NSF base filename with the extension replaced by "NDX". The newer NDX is updated within the last 15 minutes, and named like the NSF base filename, plus "_nsf", then the "NDX" extension. E.g. A database file "Arc2001.nsf" has both "Arc2001.ndx" (older) a

In [ ]:
from transformers import DataCollatorWithPadding
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(classes),
   id2label=id2class, label2id=class2id,
  problem_type = "multi_label_classification")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at mmukh/SOBertBase and are newly initialized: ['bert.embeddings.token_type_embeddings.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(50048, 768, padding_idx=0)
      (position_embeddings): Embedding(2048, 768)
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [ ]:
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=10,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_valid_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.405196,0.828210,0.159292,0.562500,0.092784
2,No log,0.345134,0.851718,0.378788,0.714286,0.257732
3,0.353500,0.325799,0.858951,0.512500,0.650794,0.422680
4,0.353500,0.333857,0.869801,0.566265,0.681159,0.484536
5,0.353500,0.328696,0.882459,0.619883,0.716216,0.546392
6,0.194900,0.342689,0.862568,0.568182,0.632911,0.515464
7,0.194900,0.342274,0.871609,0.594286,0.666667,0.536082
8,0.194900,0.349892,0.860759,0.569832,0.621951,0.525773
9,0.119400,0.349823,0.862568,0.573034,0.629630,0.525773
10,0.119400,0.350718,0.862568,0.577778,0.626506,0.536082


TrainOutput(global_step=1770, training_loss=0.20214011520989197, metrics={'train_runtime': 221.305, 'train_samples_per_second': 31.856, 'train_steps_per_second': 7.998, 'total_flos': 1296014251952850.0, 'train_loss': 0.20214011520989197, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2929232716560364,
 'eval_accuracy': 0.8806509945750453,
 'eval_f1': 0.6071428571428571,
 'eval_precision': 0.7183098591549296,
 'eval_recall': 0.5257731958762887,
 'eval_runtime': 0.703,
 'eval_samples_per_second': 112.377,
 'eval_steps_per_second': 28.45,
 'epoch': 10.0}

In [ ]:
model.save_pretrained('/content/drive/My Drive/AiExpertCource/pj/intent/intent_sobert_base')
tokenizer.save_pretrained('/content/drive/My Drive/AiExpertCource/pj/intent/intent_sobert_base')

('/content/drive/My Drive/AiExpertCource/pj/intent/intent_sobert_base/tokenizer_config.json',
 '/content/drive/My Drive/AiExpertCource/pj/intent/intent_sobert_base/special_tokens_map.json',
 '/content/drive/My Drive/AiExpertCource/pj/intent/intent_sobert_base/tokenizer.json')